In [5]:
import numpy as np
# def linespace (delta):#
#     # 生成气象版本的linspace,生成的是经纬度格网的顶点，经度从-180到180-deltaLon 维度从-90到90 所以对于0.25°的格网来说，经度是从-180到179.75 维度是从-90到89.75
#     # 纬度格网数为721 经度格网数为1440
#     lon = np.arange(-180+delta,180,delta)
#     # 按照地理坐标系的规则 维度从下到上是增大
#     lat = np.arange(90-delta,-90,-delta)
#     print('lon',lon.shape )
#     # print('lat',lat )
#     # print('lat',lat.shape )
    

#     return lon,lat

def linespace (delta):#
    # 生成气象版本的linspace,生成的是经纬度格网的顶点，经度从-180到180-deltaLon 维度从-90到90 所以对于0.25°的格网来说，经度是从-180到179.75 维度是从-90到89.75
    # 纬度格网数为721 经度格网数为1440
    lon = np.arange(0,360,delta)
    # 按照地理坐标系的规则 维度从下到上是增大
    lat = np.arange(90,-90-delta,-delta)
    print('lon',lon.shape)
    print('lat',lat.shape)
    return lon,lat
a= linespace(0.5)

lon (720,)
lat (361,)


In [3]:
import xarray as xr
data_path = '/media/dls/WeatherData/ENS10/meanstd_pl'
ds_mean = xr.open_dataset(f"{data_path}/ENS10_pl_mean.nc", chunks={"time": 10})


In [6]:
ds_mean_array=ds_mean.drop_vars('time_bnds') 
ds_mean_array.to_array()[0,0,...].values.shape

(2, 361, 720)

In [6]:
ds_mean

<xarray.Dataset>
Dimensions:    (time: 2099, bnds: 2, lon: 720, lat: 361, plev: 2)
Coordinates:
  * time       (time) datetime64[ns] 1998-01-03 1998-01-06 ... 2017-12-29
  * lon        (lon) float64 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5
  * lat        (lat) float64 90.0 89.5 89.0 88.5 ... -88.5 -89.0 -89.5 -90.0
  * plev       (plev) float64 5e+04 8.5e+04
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(10, 2), meta=np.ndarray>
    Z          (time, plev, lat, lon) float32 dask.array<chunksize=(10, 2, 361, 720), meta=np.ndarray>
    T          (time, plev, lat, lon) float32 dask.array<chunksize=(10, 2, 361, 720), meta=np.ndarray>
    Q          (time, plev, lat, lon) float32 dask.array<chunksize=(10, 2, 361, 720), meta=np.ndarray>
    W          (time, plev, lat, lon) float32 dask.array<chunksize=(10, 2, 361, 720), meta=np.ndarray>
    D          (time, plev, lat, lon) float32 dask.array<chunksize=(10, 2, 361, 720), meta=np.ndarray>
    U          (time, plev, lat, lon) float32 dask.array<chunksize=(10, 2, 361, 720), meta=np.ndarray>
    V          (time, plev, lat, lon) float32 dask.array<chunksize=(10, 2, 361, 720), meta=np.ndarray>
Attributes:
    CDI:                       Climate Data Interface version 2.0.4 (https://...
    Conventions:               CF-1.6
    institution:               European Centre for Medium-Range Weather Forec...
    frequency:                 day
    history:                   Sat Sep 16 18:25:23 2023: cdo -P 16 -f nc4 -z ...
    CDO:                       Climate Data Operators version 2.0.4 (https://...
    cdo_openmp_thread_number:  16

In [9]:
ds_mean.sel(plev =80000)

KeyError: "not all values found in index 'plev'. Try setting the `method` keyword argument (example: method='nearest')."

In [2]:
ds_mean_avg = ds_mean.mean(dim='time')
ds_mean_std = ds_mean.std(dim='time')



In [ ]:
ds_mean_std.to_netcdf(f"{data_path}/ENS10_pl_mean_std.nc")
ds_mean_avg.to_netcdf(f"{data_path}/ENS10_pl_mean_avg.nc")


In [3]:
import torch 
time= 2992
lon=720
lat=361
plev=2
lon*lat
# x =torch.randn(time,plev,lon,lat)

259920

In [4]:
import torch.nn as nn
# unfold = nn.Unfold(kernel_size=(3, 3),padding=1)
input = torch.randn(plev,time, lat, lon)#time纬度滑块展开
# output = unfold(input)
output = torch.var(input, dim=1)
# each patch contains 30 values (2x3=6 vectors, each of 5 channels)
# 4 blocks (2x3 kernels) in total in the 3x4 input
output.size()

# # Convolution is equivalent with Unfold + Matrix Multiplication + Fold (or view to output shape)
# inp = torch.randn(1, 3, 10, 12)
# w = torch.randn(2, 3, 4, 5)
# inp_unf = torch.nn.functional.unfold(inp, (4, 5))
# out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
# out = torch.nn.functional.fold(out_unf, (7, 8), (1, 1))
# # or equivalently (and avoiding a copy),
# # out = out_unf.view(1, 2, 7, 8)
# (torch.nn.functional.conv2d(inp, w) - out).abs().max()

torch.Size([2, 361, 720])

In [6]:
import xarray as xr
data_path = '/media/dls/WeatherData/ENS10/meanstd'
ds_sfc = xr.open_dataset(f"/media/dls/WeatherData/ENS10/meanstd/ENS10_sfc_mean.nc", chunks={"time": 10})



In [7]:
ds_sfc

<xarray.Dataset>
Dimensions:    (time: 2099, bnds: 2, lon: 720, lat: 361)
Coordinates:
  * time       (time) datetime64[ns] 1998-01-03 1998-01-06 ... 2017-12-29
  * lon        (lon) float64 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5
  * lat        (lat) float64 90.0 89.5 89.0 88.5 ... -88.5 -89.0 -89.5 -90.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(10, 2), meta=np.ndarray>
    SSTK       (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    TCW        (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    TCWV       (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    CP         (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    MSL        (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    TCC        (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    U10M       (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    V10M       (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    T2M        (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    TP         (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
    SKT        (time, lat, lon) float32 dask.array<chunksize=(10, 361, 720), meta=np.ndarray>
Attributes:
    CDI:                       Climate Data Interface version 2.0.4 (https://...
    Conventions:               CF-1.6
    institution:               European Centre for Medium-Range Weather Forec...
    frequency:                 day
    history:                   Tue Aug 15 19:49:08 2023: cdo -P 16 -f nc4 -z ...
    CDO:                       Climate Data Operators version 2.0.4 (https://...
    cdo_openmp_thread_number:  16

In [4]:
import numpy as np  
path = '/media/dls/WeatherData/ENS10/data/t2m/img/t2m_19980103.npy'
labelpath = '/media/dls/WeatherData/ENS10/data/t2m/label/t2mlabel_19980103.npy'

data =np.load(labelpath,allow_pickle=True)


In [12]:
data

array(<PixelData(

    META INFORMATION

    DATA FIELDS
    data: tensor([[[251.8372, 251.8372, 251.8372,  ..., 251.8372, 251.8372, 251.8372],
                 [250.6165, 250.6223, 250.6321,  ..., 250.5989, 250.6067, 250.6145],
                 [250.4153, 250.4095, 250.4036,  ..., 250.4544, 250.4407, 250.4270],
                 ...,
                 [249.2278, 249.2044, 249.1790,  ..., 249.2903, 249.2688, 249.2493],
                 [249.0559, 249.0442, 249.0305,  ..., 249.0930, 249.0794, 249.0657],
                 [248.6223, 248.6223, 248.6223,  ..., 248.6223, 248.6223, 248.6223]]])
) at 0x7f4cec82fa30>, dtype=object)

In [4]:
np.load(path,allow_pickle=False)


array([[[2.7146085e+02, 2.7146085e+02, 2.7146085e+02, ...,
         2.7146085e+02, 2.7146085e+02, 2.7146085e+02],
        [2.7146045e+02, 2.7146045e+02, 2.7146045e+02, ...,
         2.7146045e+02, 2.7146045e+02, 2.7146045e+02],
        [2.7146045e+02, 2.7146045e+02, 2.7146045e+02, ...,
         2.7146045e+02, 2.7146045e+02, 2.7146045e+02],
        ...,
        [2.7316260e+02, 2.7316357e+02, 2.7316455e+02, ...,
         2.7316064e+02, 2.7316162e+02, 2.7316162e+02],
        [2.7321436e+02, 2.7321533e+02, 2.7321631e+02, ...,
         2.7321045e+02, 2.7321240e+02, 2.7321338e+02],
        [2.7360693e+02, 2.7360693e+02, 2.7360693e+02, ...,
         2.7360693e+02, 2.7360693e+02, 2.7360693e+02]],

       [[4.7841840e-04, 4.7841840e-04, 4.7841840e-04, ...,
         4.7841840e-04, 4.7841840e-04, 4.7841840e-04],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.000

In [9]:
##测试 ckd树
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt

dggpoints = np.random.randn(20,3)
erppoints  = np.random.randn(10,3)
dggknn = spatial.cKDTree(dggpoints)

distances,indices = dggknn.query(erppoints,k=5)
indices

array([[ 1,  8, 17, 16, 11],
       [ 3, 18, 15, 11, 13],
       [ 6, 14, 12,  5,  1],
       [12, 19,  6,  4,  1],
       [ 7, 10, 14,  5, 11],
       [17,  1, 16,  5,  4],
       [ 8,  3, 18,  1, 15],
       [16, 17,  1,  8,  4],
       [ 9, 16,  4, 17, 13],
       [ 4,  2,  5,  7, 17]])

In [10]:
distances

array([[1.02224594, 1.0437659 , 1.47379396, 1.60532397, 1.63564092],
       [0.36281378, 0.41799866, 0.62056547, 1.22747636, 1.52926749],
       [0.15414694, 1.15283338, 1.28376933, 1.38691289, 1.45646083],
       [1.11367095, 1.21062444, 1.51384983, 2.09079898, 2.16095856],
       [0.42669212, 0.49427941, 0.68483306, 0.93444474, 1.25183197],
       [0.69506818, 0.83468925, 1.0398052 , 1.13887035, 1.13967484],
       [1.17147423, 1.61612044, 1.63493855, 1.66231576, 1.90643981],
       [1.23496046, 1.48667626, 1.8003906 , 1.94164043, 2.12517863],
       [0.17334806, 1.48227948, 1.62977775, 1.72498854, 1.89967626],
       [0.87796102, 0.87861597, 1.30456959, 1.48839627, 1.51250131]])

In [11]:
dggpoints

array([[ 1.50892287, -1.50528457,  1.11878784],
       [-0.18925967, -0.80606813, -0.63133609],
       [-1.32432619,  0.44258792,  1.47419739],
       [ 1.64342226, -0.52813713, -0.14382877],
       [-0.81428675,  0.60721556, -0.13635046],
       [-0.09208386, -0.26696588,  0.40070646],
       [-1.22963888, -1.12399667,  0.42330198],
       [ 0.05865258, -0.40693689,  0.98531823],
       [ 0.45431858, -1.47878076, -1.25273999],
       [ 0.38021479,  1.67958171,  0.0219519 ],
       [ 0.18440066, -1.16032957,  1.2329388 ],
       [ 0.31897136, -1.1173178 ,  0.07746005],
       [-1.40978528, -1.28560744, -0.81963969],
       [ 1.65287236,  0.54024138,  0.43647449],
       [ 0.0236975 , -1.06166329,  0.60521906],
       [ 1.60390615, -0.40836353,  0.16548349],
       [-0.26430351,  0.97127603, -1.01719234],
       [-0.52168322,  0.49651658, -0.71494959],
       [ 1.58642101, -1.16222053, -0.21287099],
       [-1.79476932,  0.23456641, -0.94665432]])

In [14]:
for i, erppoint in enumerate(erppoints):
    print(f"erppoint {i}: {erppoint}")
    for j, idx in enumerate(indices[i]):
        dggpoint = dggpoints[idx]
        distance = distances[i][j]
        print(f"  Nearest point {j + 1} in dggpoints: {dggpoint}, Distance: {distance}")

erppoint 0: [ 0.36183783 -0.45204359 -1.41615768]
  Nearest point 1 in dggpoints: [-0.18925967 -0.80606813 -0.63133609], Distance: 1.022245939377255
  Nearest point 2 in dggpoints: [ 0.45431858 -1.47878076 -1.25273999], Distance: 1.0437659036522742
  Nearest point 3 in dggpoints: [-0.52168322  0.49651658 -0.71494959], Distance: 1.4737939563945648
  Nearest point 4 in dggpoints: [-0.26430351  0.97127603 -1.01719234], Distance: 1.6053239714917318
  Nearest point 5 in dggpoints: [ 0.31897136 -1.1173178   0.07746005], Distance: 1.6356409242823466
erppoint 1: [ 1.4373291  -0.78185237 -0.3012709 ]
  Nearest point 1 in dggpoints: [ 1.64342226 -0.52813713 -0.14382877], Distance: 0.36281378296777844
  Nearest point 2 in dggpoints: [ 1.58642101 -1.16222053 -0.21287099], Distance: 0.4179986562904022
  Nearest point 3 in dggpoints: [ 1.60390615 -0.40836353  0.16548349], Distance: 0.6205654672207569
  Nearest point 4 in dggpoints: [ 0.31897136 -1.1173178   0.07746005], Distance: 1.2274763566083806


In [21]:
# scipy官方例子
import numpy as np
from scipy.spatial import cKDTree
x, y = np.mgrid[0:5, 2:8]
tree = cKDTree(np.c_[x.ravel(), y.ravel()])
# x.shape
np.c_[x.ravel(), y.ravel()].shape

(30, 2)

In [22]:
y.shape

(5, 6)

In [23]:
quary_point = np.array([[0, 0], [2.2, 2.9]])
print(quary_point.shape)
dd, ii = tree.query(quary_point, k=1)

(2, 2)


In [24]:
dd

array([2.       , 0.2236068])

In [25]:
ii

array([ 0, 13])

In [38]:
import torch
pred_stddev  = torch.randn([3,4])
pred_mean = torch.randn([3,4])
target= torch.randn([3,4])
eps =1e-15
pred_stddev =torch.zeros_like(pred_stddev)
print('pred_stddev',pred_stddev)
# pred_stddev = torch.sqrt(torch.exp(pred_stddev)) 
normed_diff = (pred_mean - target +  eps) /  (pred_stddev   +  eps) 
print('normed_diff',normed_diff)

_normal_dist = torch.distributions.Normal(0., 1.)
_frac_sqrt_pi = torch.tensor(1 / np.sqrt(np.pi))
try:
    cdf =  _normal_dist.cdf(normed_diff)
    pdf =  _normal_dist.log_prob(normed_diff).exp()
except ValueError:
    print(' ERROR CrpsGaussianLoss normed_diff',normed_diff)
    print(' ERROR CrpsGaussianLoss normed_diff',pred_mean)
    
    print(' ERROR CrpsGaussianLoss normed_diff',pred_stddev)
    
    raise ValueError

pred_stddev tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
normed_diff tensor([[-2.1199e+14, -2.3943e+15, -1.1685e+14,  6.4784e+14],
        [ 1.5058e+15,  4.8958e+14,  6.9536e+14, -1.2911e+14],
        [-7.7389e+14, -1.4065e+15,  3.8292e+14,  2.8011e+15]])


In [13]:
import xarray as xr 
data_path = '/media/dls/WeatherData/ENS10/meanstd/'
ds_mean= xr.open_dataset(f"{data_path}/ENS10_pl_mean.nc", chunks={"time": 10}).isel(time=0)
ds_mean_t850 =ds_mean.sel(plev=85000).T

In [25]:
###验证提取的 t850outvarm_19980103.npy  是否和t850_19980103.npy中的 index= 1的相同
# self.variables = ["Z", "T", "Q", "W", "D", "U", "V"]
import numpy as np
outvarm = np.load('/media/dls/WeatherData/ENS10/data/outvarmeanstd/t850/outvarmeanstd/t850outvarm_19980103.npy',allow_pickle=True)
outvars = np.load('/media/dls/WeatherData/ENS10/data/outvarmeanstd/t850/outvarmeanstd/t850outvars_19980103.npy',allow_pickle=True)
outvarsz500 = np.load('/media/dls/WeatherData/ENS10/data/outvarmeanstd/z500/outvarmeanstd/z500outvars_20171215.npy',allow_pickle=True)
outvarmz500 = np.load('/media/dls/WeatherData/ENS10/data/outvarmeanstd/z500/outvarmeanstd/z500outvarm_20171215.npy',allow_pickle=True)
outvarst2m = np.load('/media/dls/WeatherData/ENS10/data/outvarmeanstd/t2m/outvarmeanstd/t2moutvars_20011103.npy',allow_pickle=True)
outvarmt2m = np.load('/media/dls/WeatherData/ENS10/data/outvarmeanstd/t2m/outvarmeanstd/t2moutvarm_20011103.npy',allow_pickle=True)



z500 = np.load('/media/dls/WeatherData/ENS10/data/z500/img/z500_20171215.npy',allow_pickle=True)


t850 = np.load('/media/dls/WeatherData/ENS10/data/t850/img/t850_19980103.npy',allow_pickle=True) #这里存的第变量的均值和方差 先均值后方差然后按照变量顺序存储

t2m = np.load('/media/dls/WeatherData/ENS10/data/t2m/img/t2m_20011103.npy',allow_pickle=True) #这里存的第变量的均值和方差 先均值后方差然后按照变量顺序存储
print (np.allclose(outvarm,t850[2]))
print(np.allclose(outvars,t850[3],atol=1e-5))
print(np.allclose(outvarmz500,z500[0],atol=1e-5))
print(np.allclose(outvarsz500,z500[1],atol=1e-5))


#self.variables = ['SSTK', 'TCW', 'TCWV', 'CP', 'MSL', 'TCC', 'U10M', 'V10M', 'T2M', 'TP', 'SKT']
print(np.allclose(outvarmt2m,t2m[16],atol=1e-8))
print(np.allclose(outvarst2m,t2m[17],atol=1e-8))

print(t2m.shape)

True
True
True
True
True
True
(22, 361, 720)


In [17]:
# 判断矩阵是否相等 
# print(t850[2])



True
True


In [8]:
print(outvar[8])


[267.7755  267.70776 267.6355  267.55933 267.47983 267.39722 267.31186
 267.22397 267.13354 267.04135 266.94702 266.85092 266.75345 266.65326
 266.5521  266.44897 266.3441  266.23724 266.1277  266.01614 265.9019
 265.78528 265.66577 265.5437  265.4185  265.29077 265.1597  265.0267
 264.89056 264.75208 264.61145 264.4687  264.32434 264.17825 264.0308
 263.88217 263.73254 263.5818  263.43042 263.27866 263.1261  262.9726
 262.81927 262.66556 262.51184 262.35754 262.20346 262.04956 261.89624
 261.74408 261.5935  261.44507 261.29877 261.1562  261.01672 260.88235
 260.75385 260.6308  260.5146  260.406   260.3052  260.21283 260.12924
 260.05463 259.9892  259.93176 259.8839  259.8437  259.81146 259.78647
 259.76733 259.75366 259.74448 259.739   259.73587 259.73413 259.73276
 259.731   259.72766 259.72318 259.71518 259.70422 259.6898  259.67102
 259.64896 259.6224  259.59213 259.55872 259.52124 259.4812  259.4388
 259.39368 259.3474  259.29993 259.2523  259.20404 259.1562  259.10913
 259.06226 